In [ ]:
# 画像の読み込み
# $bash copy_image.sh　実行後のディレクトリ構成
# ref: https://qiita.com/ba--shi/items/09f5f2f119ffbd9bb316
import os

image_list = []
label_list = []

for dir in os.listdir("images"):
    if dir == ".DS_Store":
        continue
    img_dir = "C:/Users/~/extend_scale2/" + dir
    # フォルダごとにラベル分けする
    if dir == "none":
        label = 0
    if dir == "1":
        label = 1
    elif dir == "2":
        label = 2
    elif dir == "3":
        label = 3
    elif dir == "4":
        label = 4
    elif dir == "5":
        label = 5
    for file in os.listdir(img_dir):
        if file != ".DS_Store":
            # 配列label_listに正解ラベルを追加
            label_list.append(label)
            # 配列image_listに画像の配列データを追加
            filepath = img_dir + "/" + file
            img = cv2.imread(filepath, 0)
            image = img_to_array(img)
            image_list.append(image)